In [164]:
import pandas as pd
import numpy as np

In [ ]:
#Setting up Dataframe

In [165]:
raw_data = pd.read_csv('Flight_Data.csv', parse_dates=['fl_date']) #open a csv that was extracted from postgreSQL

C:\Users\Eduardo\AppData\Local\Temp\ipykernel_12004\2317934883.py:1: DtypeWarning: Columns (26) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv('Flight_Data.csv', parse_dates=['fl_date'])


In [166]:
raw_data = raw_data.sample(frac=0.25) #sample data so that our models don't encounter a memory error

In [167]:
model_features = ['fl_date', 'mkt_carrier', 'arr_delay','dep_delay', 'tail_num', 'mkt_carrier_fl_num', 'origin',
       'dest', 'crs_dep_time', 'crs_arr_time',
       'crs_elapsed_time', 'distance']

model_df_raw = raw_data[model_features]

#set up a base model with appropriate features

In [168]:
model_df_raw['arr_delay'] = model_df_raw['arr_delay'].clip(lower=0) #remove all negative delay values from the arr_delay
                                                                    #arr_delay is our target variable

C:\Users\Eduardo\AppData\Local\Temp\ipykernel_12004\2353730469.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  model_df_raw['arr_delay'] = model_df_raw['arr_delay'].clip(lower=0)


In [171]:
columns_model = ['fl_date', 'mkt_carrier', 'arr_delay','dep_delay', 'tail_num', 'mkt_carrier_fl_num', 'origin',
       'dest', 'crs_dep_time', 'crs_arr_time',
       'crs_elapsed_time', 'distance']

#further reduction of DF columns used in training data

In [172]:
model_df = model_df_raw[columns_model] #further reduction of DF columns used in training data

In [173]:
greater_than_zero_count = model_df[model_df['arr_delay' ] > 0]

#create a new object to represent DF['arr_delay'] with values greater than 0 (index with a boolean equation)

In [ ]:
#Feature Engineering

In [174]:
df_carriers_grouped = greater_than_zero_count.groupby('mkt_carrier').agg(count_carrier_delay = ('arr_delay', 'count')).reset_index()

#find how many delays each carrier had

In [175]:
carrier_grouped_mean = model_df.groupby('mkt_carrier').agg(avg_carrier_delay = ('arr_delay', 'mean'))

#find average carrier delay

In [176]:
df_carriers_grouped = df_carriers_grouped.merge(carrier_grouped_mean,
                                    left_on='mkt_carrier', 
                                    right_on = 'mkt_carrier',
                                    how='inner')

#merge count and avg into 1 Dataframe

In [177]:
model_df = model_df.merge(df_carriers_grouped,
                                    left_on='mkt_carrier', 
                                    right_on = 'mkt_carrier',
                                    how='inner')

#merge engineered feature with orignal Dataframe

In [178]:
df_origin_grouped = greater_than_zero_count.groupby('origin').agg(count_origin_delay = ('arr_delay', 'count')).reset_index()

#find how many delays by origin airport

In [179]:
origin_grouped_mean = model_df.groupby('origin').agg(avg_origin_delay = ('arr_delay', 'mean')).reset_index()

#find average delay by origin

In [180]:
df_origin_grouped = df_origin_grouped.merge(origin_grouped_mean,
                                    left_on='origin', 
                                    right_on = 'origin',
                                    how='inner')

#merging mean,sum, and count into a single DF

In [181]:
model_df = model_df.merge(df_origin_grouped,
                                    left_on='origin', 
                                    right_on = 'origin',
                                    how='inner')

#merge count and avg into original Dataframe

In [182]:
df_dest_grouped = greater_than_zero_count.groupby('dest').agg(count_dest_delay = ('arr_delay', 'count')).reset_index()

#find how many delays by origin airport

In [183]:
dest_grouped_mean = model_df.groupby('dest').agg(avg_dest_delay = ('arr_delay', 'mean')).reset_index()

#find average destination delay

In [184]:
df_dest_grouped = df_dest_grouped.merge(dest_grouped_mean,
                                    left_on='dest', 
                                    right_on = 'dest',
                                    how='inner')

#merge count and average into 1 Dataframe

In [185]:
model_df = model_df.merge(df_dest_grouped,
                                    left_on='dest', 
                                    right_on = 'dest',
                                    how='inner')

#merge count and average into original Dataframe       

In [186]:
df_route_group = greater_than_zero_count.groupby('mkt_carrier_fl_num').agg(count_route_delay = ('arr_delay', 'count')).reset_index()

#find how many delays by route

In [187]:
route_group_mean = model_df.groupby('mkt_carrier_fl_num').agg(avg_route_delay = ('arr_delay', 'mean')).reset_index()

#find average delay by route

In [188]:
df_route_group = df_route_group.merge(route_group_mean,
                                    left_on='mkt_carrier_fl_num', 
                                    right_on = 'mkt_carrier_fl_num',
                                    how='inner')

#merge count and average delay into one Dataframe

In [189]:
model_df = model_df.merge(df_route_group,
                                    left_on='mkt_carrier_fl_num', 
                                    right_on = 'mkt_carrier_fl_num',
                                    how='inner')

#merge count and avergage into original Dataframe

In [190]:
df_tailnum_grouped = greater_than_zero_count.groupby('tail_num').agg(count_tailnum_delay = ('arr_delay', 'count')).reset_index()

#find how many delays by plane

In [191]:
tailnum_grouped_mean = model_df.groupby('tail_num').agg(avg_tailnum_delay = ('arr_delay', 'mean')).reset_index()

#find average delay by plane

In [192]:
df_tailnum_grouped = df_tailnum_grouped.merge(tailnum_grouped_mean,
                                    left_on='tail_num', 
                                    right_on = 'tail_num',
                                    how='inner')

#merge count and average into a single Dataframe

In [193]:
model_df = model_df.merge(df_tailnum_grouped,
                                    left_on='tail_num', 
                                    right_on = 'tail_num',
                                    how='inner')

#merge count and average into original Dataframe

In [194]:
df_distance_grouped = greater_than_zero_count.groupby('distance').agg(count_delay_distance = ('arr_delay', 'count')).reset_index()

#find how many delays by distance

In [195]:
distance_grouped_mean = model_df.groupby('distance').agg(avg_delay_distance = ('arr_delay', 'mean')).reset_index()

#find average delay by distance

In [196]:
df_distance_grouped = df_distance_grouped.merge(distance_grouped_mean,
                                    left_on='distance', 
                                    right_on = 'distance',
                                    how='inner')

#merge count and average delay into a single Dataframe

In [197]:
model_df = model_df.merge(df_distance_grouped,
                                    left_on='distance', 
                                    right_on = 'distance',
                                    how='inner')

#merge count and average delay into original Dataframe

In [198]:
model_df.head(5) #check to make sure all the Dataframes merged correctly

,fl_date,mkt_carrier,arr_delay,dep_delay,tail_num,mkt_carrier_fl_num,origin,dest,crs_dep_time,crs_arr_time,...,count_origin_delay,avg_origin_delay,count_dest_delay,avg_dest_delay,count_route_delay,avg_route_delay,count_tailnum_delay,avg_tailnum_delay,count_delay_distance,avg_delay_distance
0,2019-11-24,AA,0.0,-7.0,N203NN,6073,SAN,LAX,1900,2001,...,792,10.626684,2143,13.132284,3,1.083333,13,3.022727,121,10.824356
1,2019-11-24,AA,0.0,-7.0,N203NN,6073,LAX,SAN,1736,1825,...,1997,11.600000,860,10.752715,3,1.083333,13,3.022727,121,10.824356
2,2019-04-30,AA,0.0,-7.0,N203NN,6053,LAX,SAN,1845,1935,...,1997,11.600000,860,10.752715,3,2.409091,13,3.022727,121,10.824356
3,2019-09-29,AA,1.0,11.0,N208AN,6073,SAN,LAX,1950,2046,...,792,10.626684,2143,13.132284,3,1.083333,10,9.617021,121,10.824356
4,2019-09-29,AA,0.0,-9.0,N208AN,6062,SAN,LAX,620,719,...,792,10.626684,2143,13.132284,9,9.428571,10,9.617021,121,10.824356


In [199]:
model_df.isnull().sum() #check for null values

fl_date                    0
mkt_carrier                0
arr_delay               3221
dep_delay               2670
tail_num                   0
mkt_carrier_fl_num         0
origin                     0
dest                       0
crs_dep_time               0
crs_arr_time               0
crs_elapsed_time           0
distance                   0
count_carrier_delay        0
avg_carrier_delay          0
count_origin_delay         0
avg_origin_delay           0
count_dest_delay           0
avg_dest_delay             0
count_route_delay          0
avg_route_delay            0
count_tailnum_delay        0
avg_tailnum_delay          0
count_delay_distance       0
avg_delay_distance         0
dtype: int64

In [200]:
model_df.fillna(0, inplace=True) #fill null values

In [201]:
model_df.shape #understand shape for modeling purposes

(195694, 24)

In [202]:
model_df.to_csv('final model.csv')

#save engineered Dataframe

In [203]:
X = model_df
y = model_df['arr_delay']

#X,y split - arr_delay is target column

In [204]:
X = X.drop(['fl_date','arr_delay','dep_delay','mkt_carrier','origin','dest','tail_num','mkt_carrier_fl_num'], axis=1)

#drop categorical columns

In [205]:
X.shape #review shape for modeling

(195694, 16)

In [206]:
y.shape #review shape for modeling

(195694,)

In [ ]:
#Training a Model - we are using continuous variables

In [207]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

In [208]:
scaler = MinMaxScaler()
scaler.fit_transform(X)

#setting up MinMax Scale and transforming training data

array([[0.80534351, 0.8336807 , 0.02576996, ..., 0.01220855, 0.22900763,
        0.04120772],
       [0.73579304, 0.7603168 , 0.01822753, ..., 0.01220855, 0.22900763,
        0.04120772],
       [0.78201866, 0.80616924, 0.01885607, ..., 0.01220855, 0.22900763,
        0.04120772],
       ...,
       [0.82273113, 0.02417674, 0.11187932, ..., 0.02940832, 0.        ,
        0.00193324],
       [0.43426633, 0.62692789, 0.23884349, ..., 0.05492899, 0.        ,
        0.00902178],
       [0.50848176, 0.80450188, 0.19547454, ..., 0.01173388, 0.        ,
        0.00902178]])

In [209]:
X.shape, y.shape

((195694, 16), (195694,))

In [210]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

#train test split

In [211]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((136985, 16), (58709, 16), (136985,), (58709,))

In [212]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor



import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix,auc,roc_auc_score
from sklearn.metrics import recall_score, precision_score, accuracy_score, f1_score, mean_squared_error

In [213]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((136985, 16), (136985,), (58709, 16), (58709,))

In [214]:
reg = LinearRegression().fit(X_train, y_train)
Rsquared = reg.score(X_test,y_test)

print(Rsquared)

#Linear Regression Model

0.09347236348911914


In [215]:
rfc = RandomForestRegressor()
rfc.fit(X_train, y_train)
performance = rfc.score(X_test,y_test)
print(performance)

# Random Forest Regressor Model

0.07577107886615131


In [ ]:
#Predict Delays using flight test data

#since original training data included arr_delay but test data doesnt we had to manually engineer test features rather than simply run it through previous work

In [216]:
test_data_raw = pd.read_csv('submission_csv_test_data.csv', parse_dates=['fl_date'])

In [217]:
test_data_columns = ['fl_date', 'mkt_carrier', 'tail_num', 'mkt_carrier_fl_num', 'origin',
       'dest', 'crs_dep_time', 'crs_arr_time',
       'crs_elapsed_time', 'distance']

test_df = test_data_raw[test_data_columns]

In [218]:
test_df = test_df.merge(df_carriers_grouped,
                                    left_on='mkt_carrier', 
                                    right_on = 'mkt_carrier',
                                    how='inner')

In [219]:
test_df = test_df.merge(df_origin_grouped,
                                    left_on='origin', 
                                    right_on = 'origin',
                                    how='inner')

In [220]:
test_df = test_df.merge(df_dest_grouped,
                                    left_on='dest', 
                                    right_on = 'dest',
                                    how='inner')

In [221]:
test_df = test_df.merge(df_route_group,
                                    left_on='mkt_carrier_fl_num', 
                                    right_on = 'mkt_carrier_fl_num',
                                    how='inner')

In [222]:
test_df = test_df.merge(df_tailnum_grouped,
                                    left_on='tail_num', 
                                    right_on = 'tail_num',
                                    how='inner')

In [223]:
test_df = test_df.merge(df_distance_grouped,
                                    left_on='distance', 
                                    right_on = 'distance',
                                    how='inner')

In [224]:
test_df.head(5)

,fl_date,mkt_carrier,tail_num,mkt_carrier_fl_num,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,...,count_origin_delay,avg_origin_delay,count_dest_delay,avg_dest_delay,count_route_delay,avg_route_delay,count_tailnum_delay,avg_tailnum_delay,count_delay_distance,avg_delay_distance
0,2020-01-01,WN,N951WN,5888,ONT,SFO,1810,1945,95,363,...,155,7.768786,1753,20.710723,7,16.846154,20,8.854167,91,10.071186
1,2020-01-01,WN,N467WN,6276,ONT,SFO,1150,1320,90,363,...,155,7.768786,1753,20.710723,4,14.555556,16,11.125000,91,10.071186
2,2020-01-01,WN,N7885A,4598,ONT,SJC,2020,2130,70,333,...,155,7.768786,618,11.251019,10,19.724138,18,6.254902,56,16.545455
3,2020-01-01,WN,N551WN,4761,ONT,SJC,1340,1455,75,333,...,155,7.768786,618,11.251019,17,19.259259,16,19.297297,56,16.545455
4,2020-01-01,WN,N968WN,5162,ONT,SJC,915,1035,80,333,...,155,7.768786,618,11.251019,12,12.424242,17,8.523810,56,16.545455


In [225]:
test_df = test_df.drop(['fl_date','mkt_carrier','origin','dest','tail_num','mkt_carrier_fl_num'], axis=1)

In [226]:
test_df.shape

(191, 16)

In [227]:
scaler.fit_transform(test_df) #scaling

array([[0.74052478, 0.82905983, 0.1627907 , ..., 0.24271295, 0.42307692,
        0.19923302],
       [0.35568513, 0.56196581, 0.13953488, ..., 0.3307638 , 0.42307692,
        0.19923302],
       [0.86297376, 0.90811966, 0.04651163, ..., 0.14192727, 0.25480769,
        0.43386834],
       ...,
       [0.75801749, 0.91025641, 0.25581395, ..., 0.33970352, 0.22115385,
        0.33565424],
       [0.27988338, 0.68803419, 0.81395349, ..., 0.47387301, 0.19711538,
        0.15944118],
       [0.27988338, 0.69444444, 0.88372093, ..., 0.2718674 , 0.02884615,
        0.43881032]])

In [228]:
linear_predict =  reg.predict(test_df) #linear prediction

In [229]:
RFR_predict = rfc.predict(test_df) #random forest regressor prediction

In [230]:
print(RFR_predict)

[ 23.72        27.01        16.          40.37         3.97
   3.13        18.26         7.33        21.3          4.29
   1.33        18.37         5.26        11.02         5.8
   9.76         1.99        16.46         6.53         3.52
   8.7         21.45        25.34        19.91        15.53
  10.27        31.23         2.63        33.8         37.52
   1.24         9.97         1.21         1.44        12.49
  14.99         1.34        27.32         4.8          5.21
   2.37         6.58        14.42        43.38         9.18
   8.93         1.92        11.73        17.55         7.87
 181.94         7.41         5.685        7.55        45.17
  14.35         2.1         11.9         11.34         8.77
  16.45        17.12         4.22         4.49        12.95
   9.74         6.19         9.44         6.8         10.04
  16.46         3.21         5.05        22.38         4.56
  27.19         3.46         3.29        23.84        42.17
   7.585       11.19         4.52        

In [231]:
RFR_predict_columns = ['predicted_delay']
RFR_predict_DF = pd.DataFrame(RFR_predict, columns=RFR_predict_columns)

In [232]:
RFR_predict_DF.head(5)

,predicted_delay
0,23.72
1,27.01
2,16.00
3,40.37
4,3.97


In [233]:
RFR_predict_DF = RFR_predict_DF.clip(lower=0)

In [234]:
RFR_predict_DF

,predicted_delay
0,23.72
1,27.01
2,16.00
3,40.37
4,3.97
5,3.13
6,18.26
7,7.33
8,21.30
9,4.29


In [244]:
#merged prediction delay information with submission CSV using excel

((191, 1), (191, 16))